In [1]:
import pandas as pd
import numpy as np

In [3]:
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)


In [8]:
df.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [ ]:
survivors = df["Survived"].value_counts()[1]
print(f"Number of survivors: {survivors}")

Numer of survivors: 342


## mean age

In [ ]:
#avg age based on pclass
round(df.groupby("Pclass")["Age"].mean())

Pclass
1    38.0
2    30.0
3    25.0
Name: Age, dtype: float64

In [15]:
a = round(df.groupby("Pclass")["Age"].mean())

for index,b in enumerate(a):
    print(f"Pclass: {index+1}, avg age: {b}")

Pclass: 1, avg age: 38.0
Pclass: 2, avg age: 30.0
Pclass: 3, avg age: 25.0


## survived

In [26]:
survival_by_sex = df.groupby("Sex")["Survived"].mean().round(3)
print(survival_by_sex)

Sex
female    0.742
male      0.189
Name: Survived, dtype: float64


In [18]:
df["Sex"].value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [17]:
df.groupby("Sex")["Survived"].sum()

Sex
female    233
male      109
Name: Survived, dtype: int64

In [22]:
pct_male_survivors = df[(df["Sex"] == "male") & df["Survived"] == 1].shape[0] / df[df["Sex"] == "male"].shape[0]
pct_female_survivors = df[(df["Sex"] == "female") & df["Survived"] == 1].shape[0] / df[df["Sex"] == "female"].shape[0]

In [25]:
print(f"Survivors pct:\nMale: {round(pct_male_survivors,3)}\nFemale: {round(pct_female_survivors,3)}")

Survivors pct:
Male: 0.189
Female: 0.742
